In [1]:
# =============================================================================
# Biblioteca
# =============================================================================
import time     # Facilita desenvolvimento envolvendo tempo, tempo de espera, etc
import pandas as pd
import glob # busca de arquivos em um diretório usando padrões de nome de arquivo.
import os
import shutil #Para mover um arquivo para outra pasta, você pode usar a função shutil.move() do módulo shutil.
from datetime import datetime
import sys
import unidecode #acento 
import np
#import numpy as np
import numpy as np
from hdbcli import dbapi

# pip install hdbcli --proxy=http://array

In [2]:
# =============================================================================
# Funções internas
# =============================================================================
sys.path.insert(1, '../wfuncoes py')
import keys as wk
import excel as we
import hana as wh
log = 1

### Baixar os arquivos GA.txt e GB.txt no SAP e salvar na pasta
#### Grupo A
- EANLH
- WEL_INST2
- Classe de cálculo: A
- Salvar GA.txt

#### Grupo B
- EANLH
- WEL_INST2
- Classe de cálculo: B 
- Mru: +++++T*
    - Alternativa para SAP lento: 
    - SELECT ZCGINSTAL INSTALACAO FROM CLB961851.PLANILHAO_CLIENTE WHERE SEQCC= 1 AND SUBSTRING(ZCGUNLEIT,6,1) = 'T'AND LEFT(ZCGTARTYP,1) = 'B'
- Salvar GB.txt

In [ ]:
# Carregar arquivo GA
path = "C:\\Users\\u461539\\OneDrive - IBERDROLA S.A\\005.Scripts\\ga_cliente\\INPUT\\1.GA.txt"
df = pd.read_csv(path, delimiter='|', skiprows=3, dtype={'Instalação': str, 'UnLeit. ': str})
df.columns = df.columns.str.strip() # Remover espaços em branco dos nomes das colunas
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 8','Válido até']) # Remover colunas indesejadas
df = df.dropna(how='all') #excluir NaN
df['Instalação'] = df['Instalação'].astype(int) # coluna instalacao inteiro
df['Vál.desde'] = pd.to_datetime(df['Vál.desde'], format='%d.%m.%Y', errors='coerce')

result = df

# Carregar arquivo GB
path = "C:\\Users\\u461539\\OneDrive - IBERDROLA S.A\\005.Scripts\\ga_cliente\\INPUT\\2.GB.txt"
df = pd.read_csv(path, delimiter='|', skiprows=3, dtype={'Instalação': str, 'UnLeit. ': str})
df.columns = df.columns.str.strip() # Remover espaços em branco dos nomes das colunas
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 8','Válido até']) # Remover colunas indesejadas
df = df.dropna(how='all') #excluir NaN
df['Instalação'] = df['Instalação'].astype(int) # coluna instalacao inteiro
df['Vál.desde'] = pd.to_datetime(df['Vál.desde'], format='%d.%m.%Y', errors='coerce')

result = pd.concat([result, df], ignore_index=True) # Adicionar as linhas
result = result.apply(lambda x: x.str.strip() if x.dtype == "object" else x) # Remover espaços após as palavras
result.columns = [unidecode.unidecode(col).lower().replace('.', '') for col in result.columns] #renomear as colunas: acento, minuscula
result.insert(0, 'dt_ref', datetime.now().strftime('%Y-%m-%d')) # Adicionar a data de hoje como a primeira coluna
result.rename(columns={'valdesde': 'dt_inst_desde','ctgtar':'ctg_tar','setindus':'set_indus','clcal':'cl_cal','unleit':'mru'}, inplace=True)

#coluna MRU
def determine_mru(mru):
    if mru[5] == 'T':
        return 'Telemedido'
    elif mru[5] == 'L':
        return 'Livre'
    elif mru.startswith('PR') or mru.startswith('TR'):
        return 'Transitorio'
    return 'Manual'
result['mru_status'] = result['mru'].apply(determine_mru)

#coluna grupo Cliente
def determine_grupo(ctg_tar):
    if ctg_tar.endswith('GR') or ctg_tar.endswith('CC'):
        return 'Fronteira'
    elif ctg_tar.endswith('LV'):
        return 'Livre'
    return 'Cativo'
result['grupo_cliente'] = result['ctg_tar'].apply(determine_grupo)

result = result.sort_values(by=['grupo_cliente', 'instalacao']) #ordenar

#exportar instalações cativas para pegar a CC
export = result[result['grupo_cliente'] == 'Cativo']['instalacao'].tolist()
with open('OUTPUT\\instalacao_cativo.txt', 'w') as f:
    for instalacao in export:
        f.write(f"{instalacao}\n")

### Run CCS: CC ATIVO
- ZCV_EANLA
- WEL_INST15
- Input instalacao_cativo.txt
- Salve CC.txt

### Run CCS: Liminar
- ZCV_EANLA
- WEL_LIMINAR
- Input instalacao_cativo.txt
- Salve Liminar.txt



In [ ]:
# Carregar arquivo CC
path = "C:\\Users\\u461539\\OneDrive - IBERDROLA S.A\\005.Scripts\\ga_cliente\\INPUT\\3.CC.txt"
df = pd.read_csv(path, delimiter='|', skiprows=3, dtype={'Instalação': str,'Cta.contr.': str,'ParcNeg.  ': str})
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 5','DtSolDesl.' ]) # Remover colunas indesejadas
df.columns = df.columns.str.strip() # Remover espaços em branco dos nomes das colunas
df.columns = [unidecode.unidecode(col).lower().replace('.', '') for col in df.columns] #renomear as colunas: acento, minuscula
df = df.dropna(how='all') #excluir NaN
df.rename(columns={'ctacontr': 'conta_contrato','parcneg': 'PN'}, inplace=True)
df['instalacao'] = df['instalacao'].astype(int)
df = df.sort_values(by=['instalacao']) #ordenar

#Evitar duplicação
coluna_para_excluir = ['conta_contrato','PN']
for coluna in coluna_para_excluir:
    if coluna in result.columns:
        result = result.drop(columns=[coluna])
        
#Combinar dados
result = pd.merge(result, df, on='instalacao', how='left')
result.reset_index(drop=True, inplace=True)

#Carregar as Liminares
path = "C:\\Users\\u461539\\OneDrive - IBERDROLA S.A\\005.Scripts\\ga_cliente\\INPUT\\4.liminar.txt"
df = pd.read_csv(path, delimiter='|', skiprows=3)
df = df.drop(columns=['Unnamed: 0', 'InstPrinc.','Funç.','Unnamed: 4' ]) # Remover colunas indesejadas
df.columns = [unidecode.unidecode(col).lower().replace('.', '') for col in df.columns] #renomear as colunas: acento, minuscula
df.columns = df.columns.str.strip() # Remover espaços em branco dos nomes das colunas
df = df.dropna(how='all') #excluir NaN
df['instalacao'] = df['instalacao'].astype(int)


# Função para determinar o status
def determinar_status(row):
    if row['instalacao'] in df['instalacao'].values:
        return 'Liminar'
    elif pd.notnull(row['conta_contrato']):
        return 'Ativo'
    else:
        return 'Baixado/Sem CC'

# Aplicar a função ao dataframe para criar a nova coluna 'Status'
result['status_cliente'] = result.apply(determinar_status, axis=1)
result = result.sort_values(by=['status_cliente']) #ordenar

#exportar instalações ativas para pegar o registrador lógico 
export = result[result['status_cliente'] == 'Ativo']['instalacao'].tolist()
with open('OUTPUT\\instalacao_ativo.txt', 'w') as f:
    for instalacao in export:
        f.write(f"{instalacao}\n")

### Run CCS: Registrador Lógico
- EASTS
- WEL_REG_LOG
- Input instalacao_ativo.txt
- Salve RL.txt


In [ ]:
#Carregar Reg Lógicos
path = "C:\\Users\\u461539\\OneDrive - IBERDROLA S.A\\005.Scripts\\ga_cliente\\INPUT\\5.reglog.txt"
df = pd.read_csv(path, delimiter='|', skiprows=3, dtype={'Instalação': str,'   Nº registr.lóg.': str})
df.columns = [unidecode.unidecode(col).lower().replace('.', '') for col in df.columns] #renomear as colunas: acento, minuscula
df.columns = df.columns.str.strip() # Remover espaços em branco dos nomes das colunas
df = df.drop(columns=['unnamed: 0', 'valdesde','unnamed: 4' ]) # Remover colunas indesejadas
df.rename(columns={'no registrlog': 'reg_log'}, inplace=True)
df = df.dropna(how='all') #excluir NaN
df['instalacao'] = df['instalacao'].astype(int)

#Evitar duplicação
coluna_para_excluir = 'reg_log'
if coluna_para_excluir in result.columns:
    result = result.drop(columns=[coluna_para_excluir])
#Combinar dados
result = pd.merge(result, df, on='instalacao', how='left')
result.reset_index(drop=True, inplace=True)

# Função para determinar o status
def determinar_status(row):
    if row['status_cliente'] == 'Liminar':
        return 'Liminar'
    elif row['status_cliente'] == 'Baixado/Sem CC':
        return 'Baixado/Sem CC'
    elif (row['status_cliente'] == 'Ativo' and pd.notnull(row['reg_log'])):
        return 'Ativo'
    elif (row['status_cliente'] == 'Ativo' and pd.isnull(row['reg_log'])):
        return 'Ativo s/ RegLog'
    else:
        return 'Erro'

# Aplicar a função ao dataframe para criar a nova coluna 'Status'
result['status_cliente'] = result.apply(determinar_status, axis=1)
result = result.sort_values(by=['status_cliente']) #ordenar

#exportar instalações ativas para pegar o registrador lógico 
export = result[result['status_cliente'] == 'Ativo']['reg_log'].tolist()
with open('OUTPUT\\reg_log.txt', 'w') as f:
    for instalacao in export:
        f.write(f"{instalacao}\n")

### Run CCS: equi_log, fat_reg, em_desde
- ETDZ
- VL_EQUI
- Input reg_log.txt
- Salve equi.txt

### Run CCS: fat_calc, tp_medicao
- EZUZ
- VL_FATCALC
- Input reg_log.txt
- Salve fat_calc.txt

In [ ]:
#Carregar Equi_log, Fat_reg, EM_desde 
path = "C:\\Users\\u461539\\OneDrive - IBERDROLA S.A\\005.Scripts\\ga_cliente\\INPUT\\6.equi.txt"
df = pd.read_csv(path, delimiter='|', skiprows=3, dtype={'Equipamento       ': str, '   Nº registr.lóg.': str})
df.columns = [unidecode.unidecode(col).lower().replace('.', '') for col in df.columns] #renomear as colunas: acento, minuscula
df.columns = df.columns.str.strip() # Remover espaços em branco dos nomes das colunas
df = df.drop(columns=['unnamed: 0', 'valido ate','tpreg','ca','um','unnamed: 9' ]) # Remover colunas indesejadas
df.rename(columns={'no registrlog': 'reg_log','equipamento': 'equi_log','valdesde': 'dt_em_desde','fatreg':'fat_reg'}, inplace=True)
df['dt_em_desde'] = pd.to_datetime(df['dt_em_desde'])
df['fat_reg'] = df['fat_reg'].str.replace(',', '.').str.strip().astype(float)
df = df.dropna(how='all') #excluir NaN

#Evitar duplicação
coluna_para_excluir = ['equi_log','dt_em_desde','fat_reg']
for coluna in coluna_para_excluir:
    if coluna in result.columns:
        result = result.drop(columns=[coluna])
#Combinar dados
result = pd.merge(result, df, on='reg_log', how='left')
result.reset_index(drop=True, inplace=True)

#Carregar fat_calc, tp_medicao
path = "C:\\Users\\u461539\\OneDrive - IBERDROLA S.A\\005.Scripts\\ga_cliente\\INPUT\\7.fat_calc.txt"
df = pd.read_csv(path, delimiter='|', skiprows=3, dtype={'   Nº registr.lóg.': str})
df.columns = [unidecode.unidecode(col).lower().replace('.', '') for col in df.columns] #renomear as colunas: acento, minuscula
df.columns = df.columns.str.strip() # Remover espaços em branco dos nomes das colunas
df = df.drop(columns=['unnamed: 0', 'valido ate','unnamed: 4' ]) # Remover colunas indesejadas
df.rename(columns={'no registrlog': 'reg_log','fator calc':'fat_calc'}, inplace=True)
df = df.dropna(how='all') #excluir NaN
df['fat_calc'] = df['fat_calc'].str.replace('.', '').str.replace(',', '.').str.strip().astype(float).astype(int)
df['fat_calc'].fillna(0, inplace=True)

#Evitar duplicação
coluna_para_excluir = ['fat_calc']
for coluna in coluna_para_excluir:
    if coluna in result.columns:
        result = result.drop(columns=[coluna])
#Combinar dados
result = pd.merge(result, df, on='reg_log', how='left')
result.reset_index(drop=True, inplace=True)

for index, row in result.iterrows():
    if row['status_cliente'] == 'Ativo' and pd.isna(row['fat_calc']):
        result.at[index, 'fat_calc'] = 1
        result.at[index, 'tp_medicao'] = 'MD'
    elif row['status_cliente'] == 'Ativo' and not(pd.isna(row['fat_calc'])):
        result.at[index, 'tp_medicao'] = 'MI'

result = result.sort_values(by=['status_cliente','tp_medicao','fat_reg','instalacao']) #ordenar

#exportar equipamento lógico para pegar o detalhamento do equipamento
export = result[result['status_cliente'] == 'Ativo']['equi_log'].tolist()
with open('OUTPUT\\equi_log.txt', 'w') as f:
    for instalacao in export:
        f.write(f"{instalacao}\n")        

### Run CCS: em_fabri, em_modelo, em_ano, em_material, em_descricao
- V_EQUI
- WEL_EM
- Input equi_log.txt
- Salve equi2.txt




In [ ]:
#Carregar em_fabri, em_modelo, em_ano, em_material, em, em_descricao
path = "C:\\Users\\u461539\\OneDrive - IBERDROLA S.A\\005.Scripts\\ga_cliente\\INPUT\\8.equi_log.txt"
df = pd.read_csv(path, delimiter='|', skiprows=3, dtype={'Equipamento       ': str, 'Material': str})
df.columns = [unidecode.unidecode(col).lower().replace('.', '') for col in df.columns] #renomear as colunas: acento, minuscula
df.columns = df.columns.str.strip() # Remover espaços em branco dos nomes das colunas
df = df.drop(columns=['unnamed: 0','unnamed: 8' ]) # Remover colunas indesejadas
df.rename(columns={'equipamento':'equi_log','fabricante': 'em_fabri','modelo em': 'em_modelo','anoc':'em_ano','material': 'em_material','no serie':'em','denominacao do objeto tecnico':'em_descricao'}, inplace=True)
df = df.dropna(how='all') #excluir NaN
df['em_material'] = df['em_material'].str.strip()
df['em_fabri'] = df['em_fabri'].str.strip()
df['em_modelo'] = df['em_modelo'].str.strip()
df['em_material'] = df['em_material'].str.strip()
df['em'] = df['em'].str.strip()
df['em_descricao'] = df['em_descricao'].str.strip()
df = df.drop_duplicates()

#print(df.head())         # Exibir as primeiras linhas do DataFrame
#print(df.columns)
#Evitar duplicação
coluna_para_excluir = ['em_fabri','em_modelo','em_ano','em_material','em','em_descricao']
for coluna in coluna_para_excluir:
    if coluna in result.columns:
        result = result.drop(columns=[coluna])
#Combinar dados
result = pd.merge(result, df, on='equi_log', how='left')
print(f"O DataFrame tem {len(result)} linhas. \n")

nova_ordem_colunas = ['dt_ref', 'instalacao', 'dt_inst_desde', 'ctg_tar', 'set_indus', 'cl_cal', 'mru','mru_status','grupo_cliente','conta_contrato','PN','status_cliente','reg_log','equi_log','em','dt_em_desde','fat_reg','fat_calc','tp_medicao','em_fabri','em_modelo','em_ano','em_material','em_descricao']
result = result[nova_ordem_colunas] # Reordenar as colunas do DataFrame
result.reset_index(drop=True, inplace=True)

In [ ]:

global conn
global cursor
conn = dbapi.connect(
    address=wk.servidor_hana,
    port=wk.porta_hana,
    user=wk.user_hana,
    password=wk.pass_hana,
    databasename='BNP',
    sslValidateCertificate=False
)
cursor = conn.cursor()
if log:
    print('SAP HANA connected')

# Executar insert into
#wh.hana_insert(result, 'ga_cliente',log)
# É mais rápido fazer import pelo DBeaver
result.to_csv('OUTPUT\\result.csv', sep=';', index=False)

#exportar result
print(f"O DataFrame tem {len(result)} linhas. \n")
print(result.head())         # Exibir as primeiras linhas do DataFrame
print(result.info())         # Exibir as primeiras linhas do DataFrame
result.to_csv('OUTPUT\\result.csv', sep=';', index=False)

#exportar df
print(f"O DataFrame tem {len(df)} linhas. \n")
print(df.head())         # Exibir as primeiras linhas do DataFrame
print(df.info())         # Exibir as primeiras linhas do DataFrame
df.to_csv('OUTPUT\\df.csv', sep=';', index=False)